In [1]:
#导入数据时增加一个维度变为（10505，1，30，200），都要增加吗？
#batchsize=16的时候，是怎么放入模型的？第一个卷积输入通道为1，是（1500，1，30，1024）中的1吗？
#（1500，1，30，1024）叫3维数据还是4？不是二维卷积码？
#（8，1，1，64）解释？
#实例化类的时候，传入的参数是——init的？
#标签制作：不用独热编码的区别？
#打印的print("block1", x.shape)指的是这一层的数据的形状吗
#十折交叉验证修改一下？不知道对不对 chatgpt生成的
#classes_num=2去除的时候，EEGNet.__init__() missing 1 required positional argument: 'classes_num'


In [2]:
import random
import torch
from sklearn.model_selection import KFold
import math
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

In [3]:
# 设置GPU使用方式
# 获取GPU列表
gpus = [torch.cuda.device(i) for i in range(torch.cuda.device_count())]

if gpus:
    try:
        # 设置每个GPU为增长式占用
        for gpu in gpus:
            torch.cuda.set_per_process_memory_fraction(1.0, gpu)  # 可根据需要调整占用比例
    except RuntimeError as e:
        # 打印异常
        print(e)

In [4]:
class EEGNet(nn.Module):
    def __init__(self, classes_num):
        super(EEGNet, self).__init__()
        self.drop_out = 0.5
        
        #8个1通道的卷积核
        self.block_1 = nn.Sequential(
            # Pads the input tensor boundaries with zero
            # left, right, up, bottom
            nn.ZeroPad2d((15, 16, 0, 0)),
            nn.Conv2d(
                in_channels=1,  # input shape (1, C, T)
                out_channels=8,  # num_filters
                kernel_size=(1, 32),  # filter size（8，1，1，64）
                bias=False
            ),  # output shape (8, C, T)
            nn.BatchNorm2d(8)  # output shape (8, C, T)
        )
        
        #16个1通道卷积核，分组卷积，相当于每组1个特征图被两个卷积核卷积
        self.block_2 = nn.Sequential(
            nn.Conv2d(
                in_channels=8,  # input shape (8, C, T)
                out_channels=16,  # num_filters
                kernel_size=(30, 1),  # filter size
                groups=8,
                bias=False
            ),  # output shape (16, 1, T)
            nn.BatchNorm2d(16),  # output shape (16, 1, T)
            nn.ELU(),
            nn.AvgPool2d((1, 4)),  # output shape (16, 1, T//4)
            nn.Dropout(self.drop_out)  # output shape (16, 1, T//4)
        )
        #16个1通道卷积核，分组卷积。每组一个特征数被一个卷积核卷积
        self.block_3 = nn.Sequential(
            nn.ZeroPad2d((7, 8, 0, 0)),
            nn.Conv2d(
                in_channels=16,  # input shape (16, 1, T//4)
                out_channels=16,  # num_filters
                kernel_size=(1, 16),  # filter size
                groups=16,
                bias=False
            ),  # output shape (16, 1, T//4)
            nn.Conv2d(
                in_channels=16,  # input shape (16, 1, T//4)
                out_channels=16,  # num_filters
                kernel_size=(1, 1),  # filter size
                bias=False
            ),  # output shape (16, 1, T//4)
            nn.BatchNorm2d(16),  # output shape (16, 1, T//4)
            nn.ELU(),
            nn.AvgPool2d((1, 8)),  # output shape (16, 1, T//32)
            nn.Dropout(self.drop_out)
        )

        self.out = nn.Linear((96), classes_num)

    def forward(self,x):
        x = self.block_1(x)
        #print("block1", x.shape)
        x = self.block_2(x)
        #print("block2", x.shape)
        x = self.block_3(x)
        #print("block3", x.shape)

        x = x.view(x.size(0), -1)
        
        x = self.out(x)
        return x # return x for visualization

In [5]:
#导入数据
#rest1
datapath1=r'D:\JQ_YJS\飞行试验数据\处理后\rest1.npy' 
data1=np.load(datapath1)
#print(data1.shape)
data1=torch.tensor(data1).unsqueeze(1)
print(data1.shape)
data1=data1.numpy()

#rest2
datapath2=r'D:\JQ_YJS\飞行试验数据\处理后\rest2.npy'
data2=np.load(datapath2)
#print(data2.shape)
data2=torch.tensor(data2).unsqueeze(1)
print(data2.shape)
data2=data2.numpy()
data_all=np.concatenate((data1,data2),axis=0)
#标签制作
label_all = torch.cat([torch.zeros(6344), torch.ones(6589)]).long()  # 标签：前10505个为0，后10505个为1
print(label_all.shape)
print(data_all.shape)

torch.Size([6344, 1, 30, 200])
torch.Size([6589, 1, 30, 200])
torch.Size([12933])
(12933, 1, 30, 200)


In [6]:
# 创建十折交叉验证
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

# 保存结果的列表
historys = []
test_pred = []
test_real = []
accuracy, precision, recall, f1score = [], [], [], []
batchsz = 16
num_epochs = 100

# 进行十折交叉验证
for fold, (train_ind, test_ind) in enumerate(kfold.split(data_all, label_all)):
    print('fold号:', fold + 1)

    # 每一折验证前都要打乱训练集样本顺序
    n = len(train_ind)
    A = np.linspace(0, n - 1, n, dtype=int)
    random.shuffle(A)

    # 构建训练集、验证集、测试集
    epoch_train = data_all[train_ind[A[:int(0.8 * n)]]]
    epoch_val = data_all[train_ind[A[int(0.8 * n):]]]
    epoch_test = data_all[test_ind]
    label_train = label_all[train_ind[A[:int(0.8 * n)]]]
    label_val = label_all[train_ind[A[int(0.8 * n):]]]
    label_test = label_all[test_ind]

    # 转换为Tensor并创建DataLoader
    train_dataset = TensorDataset(torch.tensor(epoch_train, dtype=torch.float32), torch.tensor(label_train, dtype=torch.long))
    val_dataset = TensorDataset(torch.tensor(epoch_val, dtype=torch.float32), torch.tensor(label_val, dtype=torch.long))
    
    train_loader = DataLoader(train_dataset, batch_size=batchsz, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batchsz, shuffle=False)

    # 选择、创建模型
    model = EEGNet(classes_num=1)
    #print(model)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    # 配置模型训练
    criterion = nn.BCEWithLogitsLoss()  # 使用交叉熵损失
    
    #optimizer = optim.SGD(model.parameters(), lr=1e-4)
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

    # 开始训练模型
    print('开始训练!!')
    history = {'train_loss': [], 'val_loss': []}

    for epoch in range(num_epochs):  # 训练100个epoch
        model.train()
        running_loss = 0.0

        # 训练阶段
        for inputs, targets in train_loader:
            #print(f"训练批次输入数据形状: {inputs.shape}")
            inputs = inputs.to(device)
            
            targets = targets.to(device)
            optimizer.zero_grad()
            
            outputs = model(inputs)
            targets = targets.view(-1).float() 
            #targets=targets.view(-1,1).float()
            loss = criterion(outputs.view(-1), targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        history['train_loss'].append(avg_train_loss)

        #print('Epoch:{}/{}'.format(epoch+1,num_epochs))
        
        # 计算训练集的指标
        with torch.no_grad():
            train_pred = model(torch.tensor(epoch_train, dtype=torch.float32).to(device))
            train_pred = (torch.sigmoid(train_pred) > 0.5).cpu().numpy().astype(int)
            
            acc_train = accuracy_score(label_train, train_pred)
            pre_train = precision_score(label_train, train_pred, average='macro')
            rec_train = recall_score(label_train, train_pred, average='macro')
            f1_train = f1_score(label_train, train_pred, average='macro')

        # 输出训练集指标
       
        # 验证阶段
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                

                inputs = inputs.to(device)
            
                targets = targets.to(device)
                outputs = model(inputs)
                #targets=targets.view(-1,1).float()
                targets = targets.view(-1).float()
                loss = criterion(outputs.view(-1), targets)
                val_loss += loss.item()

        avg_val_loss = val_loss / len(val_loader)
        
        history['val_loss'].append(avg_val_loss)
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, VAL Loss: {avg_val_loss:.4f}')
        #print(f'Epoch [{epoch+1}/{num_epochs}], VAL Loss: {avg_val_loss:.4f}')
        print(f"$$训练集准确率 accuracy: {acc_train}",f"$$训练集精确率 precision: {pre_train}",f"$$训练集召回率 recall: {rec_train}",f"$$训练集 F1 评分 f1_score: {f1_train}")
        


        # 保存训练记录
        historys.append(history)
        

        # 计算、保存测试结果
        model.eval()
        with torch.no_grad():
            pred_test = model(torch.tensor(epoch_test, dtype=torch.float32).to(device))
            
            pred_test = (torch.sigmoid(pred_test) > 0.5).cpu().numpy().astype(int)


        # 保存预测结果和真实结果
        test_pred.append(pred_test)
        test_real.append(label_test)

        # 计算准确率，精确率，召回率，F1评分
        acc = accuracy_score(label_test, pred_test)
        pre = precision_score(label_test, pred_test, average='macro')
        rec = recall_score(label_test, pred_test, average='macro')
        f1 = f1_score(label_test, pred_test, average='macro')
        
        accuracy.append(acc)
        precision.append(pre)
        recall.append(rec)
        f1score.append(f1)
        
        print(f"@@测试集准确率 accuracy: {acc}",f"@@测试集精确率 precision: {pre}",f"@@测试集召回率 recall: {rec}",f"@@测试集 F1 评分 f1_score: {f1}")
        

# 将每一折 history 中误差结果保存（训练集和测试集，用于反映训练过程）    
loss_train = []
loss_val = []
for history_s in historys:
    loss_val.append(history_s['val_loss'])
    loss_train.append(history_s['train_loss'])

fold号: 1


C:\Users\FJQ\AppData\Local\Temp\ipykernel_21364\2905615022.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = TensorDataset(torch.tensor(epoch_train, dtype=torch.float32), torch.tensor(label_train, dtype=torch.long))
C:\Users\FJQ\AppData\Local\Temp\ipykernel_21364\2905615022.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_dataset = TensorDataset(torch.tensor(epoch_val, dtype=torch.float32), torch.tensor(label_val, dtype=torch.long))


开始训练!!
Epoch [1/100], Train Loss: 0.5547, VAL Loss: 0.5911
$$训练集准确率 accuracy: 0.7777897110944044 $$训练集精确率 precision: 0.8010083864478335 $$训练集召回率 recall: 0.78024054231035 $$训练集 F1 评分 f1_score: 0.7743959089059461
@@测试集准确率 accuracy: 0.7109737248840804 @@测试集精确率 precision: 0.786903005716393 @@测试集召回率 recall: 0.7081637041284403 @@测试集 F1 评分 f1_score: 0.6887182257898529
Epoch [2/100], Train Loss: 0.4683, VAL Loss: 0.5991
$$训练集准确率 accuracy: 0.81119106433251 $$训练集精确率 precision: 0.8263835906921857 $$训练集召回率 recall: 0.8131073257644863 $$训练集 F1 评分 f1_score: 0.8095702680554566
@@测试集准确率 accuracy: 0.7395672333848532 @@测试集精确率 precision: 0.8030741741879324 @@测试集召回率 recall: 0.7370699541284403 @@测试集 F1 评分 f1_score: 0.7238186015434258
Epoch [3/100], Train Loss: 0.4359, VAL Loss: 0.5500
$$训练集准确率 accuracy: 0.8266566426807003 $$训练集精确率 precision: 0.8361443637983452 $$训练集召回率 recall: 0.8281539585986021 $$训练集 F1 评分 f1_score: 0.8258363542806282
@@测试集准确率 accuracy: 0.7465224111282844 @@测试集精确率 precision: 0.811403990146

C:\Users\FJQ\AppData\Local\Temp\ipykernel_21364\2905615022.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = TensorDataset(torch.tensor(epoch_train, dtype=torch.float32), torch.tensor(label_train, dtype=torch.long))
C:\Users\FJQ\AppData\Local\Temp\ipykernel_21364\2905615022.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_dataset = TensorDataset(torch.tensor(epoch_val, dtype=torch.float32), torch.tensor(label_val, dtype=torch.long))


开始训练!!
Epoch [1/100], Train Loss: 0.5424, VAL Loss: 0.5988
$$训练集准确率 accuracy: 0.7914294919987112 $$训练集精确率 precision: 0.8105003512154514 $$训练集召回率 recall: 0.7936207244773872 $$训练集 F1 评分 f1_score: 0.7889709266168139
@@测试集准确率 accuracy: 0.6731066460587326 @@测试集精确率 precision: 0.7743185947910357 @@测试集召回率 recall: 0.6731066460587326 @@测试集 F1 评分 f1_score: 0.6398903411673915
Epoch [2/100], Train Loss: 0.4587, VAL Loss: 0.4970
$$训练集准确率 accuracy: 0.8190312533562453 $$训练集精确率 precision: 0.8261656150651666 $$训练集召回率 recall: 0.8203430581147815 $$训练集 F1 评分 f1_score: 0.8184113931694756
@@测试集准确率 accuracy: 0.7519319938176198 @@测试集精确率 precision: 0.8008976056241583 @@测试集召回率 recall: 0.7519319938176198 @@测试集 F1 评分 f1_score: 0.74141186504515
Epoch [3/100], Train Loss: 0.4178, VAL Loss: 0.5253
$$训练集准确率 accuracy: 0.8481366126087424 $$训练集精确率 precision: 0.8505145979498621 $$训练集召回率 recall: 0.8488816344411897 $$训练集 F1 评分 f1_score: 0.8480313937963146
@@测试集准确率 accuracy: 0.714064914992272 @@测试集精确率 precision: 0.7918372398

...

开始训练!!
Epoch [1/100], Train Loss: 0.5481, VAL Loss: 0.5844
$$训练集准确率 accuracy: 0.788659793814433 $$训练集精确率 precision: 0.809679720314501 $$训练集召回率 recall: 0.7901288397739995 $$训练集 F1 评分 f1_score: 0.7855688901754692
@@测试集准确率 accuracy: 0.7099767981438515 @@测试集精确率 precision: 0.7797529268117503 @@测试集召回率 recall: 0.7244945989692653 @@测试集 F1 评分 f1_score: 0.6987135792174352
Epoch [2/100], Train Loss: 0.4565, VAL Loss: 0.4886
$$训练集准确率 accuracy: 0.8295747422680413 $$训练集精确率 precision: 0.8406627065615752 $$训练集召回率 recall: 0.8305979596251569 $$训练集 F1 评分 f1_score: 0.8284649786019169
@@测试集准确率 accuracy: 0.7494199535962877 @@测试集精确率 precision: 0.7948588162494301 @@测试集召回率 recall: 0.7609077285891999 @@测试集 F1 评分 f1_score: 0.7442979564930785
Epoch [3/100], Train Loss: 0.4178, VAL Loss: 0.4310
$$训练集准确率 accuracy: 0.845360824742268 $$训练集精确率 precision: 0.8545970680646973 $$训练集召回率 recall: 0.8462797236717705 $$训练集 F1 评分 f1_score: 0.844575317791576
@@测试集准确率 accuracy: 0.7718484145398299 @@测试集精确率 precision: 0.80648814281

C:\Users\FJQ\AppData\Local\Temp\ipykernel_21364\2905615022.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = TensorDataset(torch.tensor(epoch_train, dtype=torch.float32), torch.tensor(label_train, dtype=torch.long))
C:\Users\FJQ\AppData\Local\Temp\ipykernel_21364\2905615022.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_dataset = TensorDataset(torch.tensor(epoch_val, dtype=torch.float32), torch.tensor(label_val, dtype=torch.long))


开始训练!!
Epoch [1/100], Train Loss: 0.5727, VAL Loss: 0.5499
$$训练集准确率 accuracy: 0.7705111683848798 $$训练集精确率 precision: 0.7976788545102114 $$训练集召回率 recall: 0.7730661441598942 $$训练集 F1 评分 f1_score: 0.7662242652630764
@@测试集准确率 accuracy: 0.6860015467904099 @@测试集精确率 precision: 0.7731761904452092 @@测试集召回率 recall: 0.7015959438826953 @@测试集 F1 评分 f1_score: 0.6687594332241309
Epoch [2/100], Train Loss: 0.4832, VAL Loss: 0.4847
$$训练集准确率 accuracy: 0.8016537800687286 $$训练集精确率 precision: 0.8214537832840176 $$训练集召回率 recall: 0.8037639092326592 $$训练集 F1 评分 f1_score: 0.7992689318417552
@@测试集准确率 accuracy: 0.7494199535962877 @@测试集精确率 precision: 0.8057395918138177 @@测试集召回率 recall: 0.761447440440181 @@测试集 F1 评分 f1_score: 0.7425644983678766
Epoch [3/100], Train Loss: 0.4350, VAL Loss: 0.5088
$$训练集准确率 accuracy: 0.8277491408934707 $$训练集精确率 precision: 0.8412093680489552 $$训练集召回率 recall: 0.8294480013230013 $$训练集 F1 评分 f1_score: 0.8265067764401968
@@测试集准确率 accuracy: 0.7362722351121423 @@测试集精确率 precision: 0.80059768

C:\Users\FJQ\AppData\Local\Temp\ipykernel_21364\2905615022.py:30: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = TensorDataset(torch.tensor(epoch_train, dtype=torch.float32), torch.tensor(label_train, dtype=torch.long))
C:\Users\FJQ\AppData\Local\Temp\ipykernel_21364\2905615022.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_dataset = TensorDataset(torch.tensor(epoch_val, dtype=torch.float32), torch.tensor(label_val, dtype=torch.long))


开始训练!!
Epoch [1/100], Train Loss: 0.5634, VAL Loss: 0.5232
$$训练集准确率 accuracy: 0.7900558419243986 $$训练集精确率 precision: 0.8117620973463886 $$训练集召回率 recall: 0.7919335916692593 $$训练集 F1 评分 f1_score: 0.7870353006171498
@@测试集准确率 accuracy: 0.7385924207269915 @@测试集精确率 precision: 0.8036241522668846 @@测试集召回率 recall: 0.7437020050173309 @@测试集 F1 评分 f1_score: 0.7263954623995192
Epoch [2/100], Train Loss: 0.4720, VAL Loss: 0.5660
$$训练集准确率 accuracy: 0.8077749140893471 $$训练集精确率 precision: 0.8249799196981764 $$训练集召回率 recall: 0.8094167592859727 $$训练集 F1 评分 f1_score: 0.8057359800029901
@@测试集准确率 accuracy: 0.6875483372003094 @@测试集精确率 precision: 0.7819862155388471 @@测试集召回率 recall: 0.6939164863363909 @@测试集 F1 评分 f1_score: 0.6632986129015624
Epoch [3/100], Train Loss: 0.4412, VAL Loss: 0.4673
$$训练集准确率 accuracy: 0.8324742268041238 $$训练集精确率 precision: 0.8420348586299407 $$训练集召回率 recall: 0.8336729515530861 $$训练集 F1 评分 f1_score: 0.8316180850692911
@@测试集准确率 accuracy: 0.7540603248259861 @@测试集精确率 precision: 0.8111401

In [7]:
# 假设 accuracy 列表包含了 1000 个准确率值
# 每一折包含 100 个 epoch
num_folds = 10
epochs_per_fold = 100

# 打印每一折的最大准确率和对应的epoch
for fold in range(num_folds):
    # 获取当前折的准确率列表
    start_idx = fold * epochs_per_fold
    end_idx = (fold + 1) * epochs_per_fold
    fold_accuracy = accuracy[start_idx:end_idx]

    # 找到最大准确率和对应的epoch
    max_acc = max(fold_accuracy)
    max_epoch = fold_accuracy.index(max_acc) + 1  # +1 因为索引从0开始

    # 打印结果
    print(f'第{fold + 1}折准确率最高为{max_acc:.4f}，对应的epoch为{max_epoch}')

第1折准确率最高为0.9621，对应的epoch为100
第2折准确率最高为0.9722，对应的epoch为91
第3折准确率最高为0.9722，对应的epoch为99
第4折准确率最高为0.9629，对应的epoch为69
第5折准确率最高为0.9783，对应的epoch为48
第6折准确率最高为0.9474，对应的epoch为80
第7折准确率最高为0.9637，对应的epoch为81
第8折准确率最高为0.9675，对应的epoch为83
第9折准确率最高为0.9660，对应的epoch为80
第10折准确率最高为0.9706，对应的epoch为98
